# Classify new texts

And put the results in a naf file.

In [9]:
! ls /home/jvdzwaan/data/tmp-naf

DB_AA.txt.xml  DB_CC.txt.xml  DB_EE.txt.xml  DB_GG.txt.xml  DB_II.txt.xml
DB_BB.txt.xml  DB_DD.txt.xml  DB_FF.txt.xml  DB_HH.txt.xml  DB_JJ.txt.xml


In [10]:
%run embem/kafnaftag/emotions_layer.py -c /home/jvdzwaan/data/embem/ml/classifier/classifier.pkl -d /home/jvdzwaan/data/embem/dict/hist2modern_bwnt.json -b /home/jvdzwaan/data/embem/dict/body_part_mapping.json /home/jvdzwaan/data/tmp-naf /home/jvdzwaan/data/output 

loading body part mapping from /home/jvdzwaan/data/embem/dict/body_part_mapping.json
ignored: rose-kaken (cheeks)

/home/jvdzwaan/data/tmp-naf/DB_GG.txt.xml (1 of 10)
writing /home/jvdzwaan/data/output/DB_GG.txt.xml

/home/jvdzwaan/data/tmp-naf/DB_EE.txt.xml (2 of 10)
writing /home/jvdzwaan/data/output/DB_EE.txt.xml

/home/jvdzwaan/data/tmp-naf/DB_JJ.txt.xml (3 of 10)
writing /home/jvdzwaan/data/output/DB_JJ.txt.xml

/home/jvdzwaan/data/tmp-naf/DB_CC.txt.xml (4 of 10)
writing /home/jvdzwaan/data/output/DB_CC.txt.xml

/home/jvdzwaan/data/tmp-naf/DB_DD.txt.xml (5 of 10)
writing /home/jvdzwaan/data/output/DB_DD.txt.xml

/home/jvdzwaan/data/tmp-naf/DB_BB.txt.xml (6 of 10)
writing /home/jvdzwaan/data/output/DB_BB.txt.xml

/home/jvdzwaan/data/tmp-naf/DB_FF.txt.xml (7 of 10)
writing /home/jvdzwaan/data/output/DB_FF.txt.xml

/home/jvdzwaan/data/tmp-naf/DB_II.txt.xml (8 of 10)
writing /home/jvdzwaan/data/output/DB_II.txt.xml

/home/jvdzwaan/data/tmp-naf/DB_HH.txt.xml (9 of 10)
writing /home/jvd

In [11]:
! ls /home/jvdzwaan/data/output/

DB_AA.txt.xml  DB_CC.txt.xml  DB_EE.txt.xml  DB_GG.txt.xml  DB_II.txt.xml
DB_BB.txt.xml  DB_DD.txt.xml  DB_FF.txt.xml  DB_HH.txt.xml  DB_JJ.txt.xml


## Load classifier and do predictions in notebook

This is the 'old' way of making predictions. There is no script for putting the predictions in the naf files.

In [1]:
# Copy required files for loading the classifier
! cp embem/machinelearning/mlutils.py utils.py
! cp embem/machinelearning/rakel.py .

In [2]:
# load classifier
from sklearn.externals import joblib
from embem.machinelearning import mlutils as utils

classifier = '/home/jvdzwaan/data/embem/ml/classifier/classifier.pkl'

clf = joblib.load(classifier)

In [9]:
# train file
embem_data_dir = '/home/jvdzwaan/data/embem/'
train_file = os.path.join(embem_data_dir, 'ml/all_spellingnormalized.txt')

In [5]:
data_dir = '/home/jvdzwaan/data/tmp/normalized/'

In [16]:
# specify the path where output should be written
out_dir = '/home/jvdzwaan/data/tmp/txt-predictions-normalized'
os.makedirs(out_dir) 

In [23]:
# Load data
import os
import codecs
from utils import get_data, load_data

text_files = [fi for fi in os.listdir(data_dir) if fi.endswith('.txt')]
for i, text_file in enumerate(text_files):
    in_file = os.path.join(data_dir, text_file)
    print('({} of {}) {}'.format(i+1, len(text_files), text_file))

    # load data
    X_train, X_data, Y_train, Y_data, classes_ = get_data(train_file, in_file)

    # classifiy
    pred = clf.predict(X_data)

    # save results
    out_file = os.path.join(out_dir, text_file)

    X_data_with_ids, Y_data = load_data(in_file)

    with codecs.open(out_file, 'wb', 'utf8') as f:
        for x, y in zip(X_data_with_ids, pred):
            f.write(u'{}\t{}\n'.format(x.decode('utf8'),
                                       '_'.join(classes_[y]) or 'None'))

(1 of 1) input.txt.xml.txt


In [24]:
! ls /home/jvdzwaan/data/tmp/txt-predictions-normalized

input.txt.xml.txt


In [25]:
! cat /home/jvdzwaan/data/tmp/txt-predictions-normalized/input.txt.xml.txt

input.txt.p.1.s.1	deze zijn een tekst .	None
input.txt.p.1.s.2	Een mooi tets .	None


In [28]:
# make unnormalized version of predicted labels (needed before expanding body part labels)

%run embem/machinelearningdata/merge_data_and_labels.py /home/jvdzwaan/data/tmp/txt-predictions-normalized /home/jvdzwaan/data/tmp/unnormalized/ /home/jvdzwaan/data/tmp/txt-predictions-unnormalized

(1 of 1) input.txt.xml.txt


In [29]:
! cat /home/jvdzwaan/data/tmp/txt-predictions-unnormalized/input.txt.xml.txt

input.txt.p.1.s.1	Dit is een tekst .	None
input.txt.p.1.s.2	Een mooie test .	None


In [ ]:
# Expand body parts

%run classify_body_parts.py /home/jvdzwaan/data/embem/dict/body_part_mapping.json /home/jvdzwaan/data/embem/txt/annotation-predicted-heem/ /home/jvdzwaan/data/embem/txt/annotation-predicted-heem-expanded_body_parts  /home/jvdzwaan/data/embem/dict/annotation_heem_expanded_body_parts.csv